## Installation

In [70]:
%pip install gym==0.21.0
%pip install numpy
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Environement Setup

In [6]:
import gym
import numpy as np
import matplotlib.pyplot as plt

In [18]:
env = gym.make('MountainCar-v0')
Qpts = None

# Training

In [8]:
def rewards_calcul(pos):
    if (pos >= 0.5):
        return 2
    else:
        return (pos + 1.2) / 1.8 - 1

In [9]:
def stateSpace(env):
    size_states = (env.observation_space.high - env.observation_space.low) * np.array([10, 50])
    size_states = np.around(size_states, 0).astype(int) + 1
    return size_states

In [10]:
def lower_state(env):
    state = env.reset()
    low_state = (state - env.observation_space.low) * np.array([10, 50])
    low_state = np.round(low_state, 0).astype(int)
    return low_state

In [11]:
def update(learningRate, nextState, nextState_adj, Q_Table, currentState, action):
    delta = learningRate * (rewards_calcul(nextState[0]) + np.max(Q_Table[nextState_adj[0], nextState_adj[1]]) - Q_Table[currentState[0], currentState[1], action])
    return delta

In [12]:
def training(env, learningrate, epsilon, min_epsilon, episodes):

    reward_list = []
    average_rewards = []    
    eps1 = epsilon
    first = episodes + 1
    size_states = stateSpace(env)
    
    Q_Table = np.random.uniform(low = -1, high = 0, size = (size_states[0], size_states[1], env.action_space.n))    
    Qinit = np.copy(Q_Table)
    
    for i in range(episodes):
        done = False
        tot_reward = 0
        reward = 0
        env.reset()
        
        state_adj = lower_state(env)
        
        while done != True:
            # Calculate next action
            if np.random.random() < 1 - epsilon:
                action = np.argmax(Q_Table[state_adj[0], state_adj[1]])
            else:
                action = np.random.randint(0, env.action_space.n)
                
            nextState, reward, done, _ = env.step(action)
            
            nextState_adj = (nextState - env.observation_space.low) * np.array([10, 50])
            nextState_adj = np.round(nextState_adj, 0).astype(int)
            
            row = np.array([state_adj[0], state_adj[1], action])
            
            # Allow for terminal state
            if done and nextState[0] >= 0.5:
                Q_Table[state_adj[0], state_adj[1], action] = reward
            # Adjust Q value for current state
            else:
                delta = update(learningrate, nextState, nextState_adj, Q_Table, state_adj, action)
                Q_Table[state_adj[0], state_adj[1], action] += delta
            
            # Say when first success occurs
            if nextState[0] >= 0.5 and i < first:
                first = i
                print('First time reaching goal on epsiode {}'.format(first + 1))
            
            tot_reward += rewards_calcul(nextState[0])
            state_adj = nextState_adj
        
        if epsilon > min_epsilon:
            epsilon *= eps1
            
        reward_list.append(tot_reward)
        
        if (i + 1) % 100 == 0:
            ave_reward = np.mean(reward_list)
            average_rewards.append(ave_reward)
            reward_list = []
            print('Episode {} Average Reward: {}'.format(i + 1, ave_reward))
        
    env.close()
    return average_rewards, Q_Table, Qinit
        

In [8]:
env.reset()
rewards, Qpts, Qinit = training(env, 0.2, 0.999, 0, 10000)
np.savetxt("data.csv", Qpts.reshape(-1, Qpts.shape[-1]), delimiter=",")

Episode 100 Average Reward: -124.87117748989
Episode 200 Average Reward: -124.69561486914752
Episode 300 Average Reward: -125.0994041423003
Episode 400 Average Reward: -124.95931698724418
Episode 500 Average Reward: -124.8774333804173
Episode 600 Average Reward: -125.0985500107222
Episode 700 Average Reward: -124.02876699250977
Episode 800 Average Reward: -124.3995555999477
Episode 900 Average Reward: -123.65755292625498
First time reaching goal on epsiode 908
Episode 1000 Average Reward: -124.57879487771594
Episode 1100 Average Reward: -123.9720963533125
Episode 1200 Average Reward: -123.66913878393156
Episode 1300 Average Reward: -122.41190176475433
Episode 1400 Average Reward: -123.04915669257852
Episode 1500 Average Reward: -122.22274414317987
Episode 1600 Average Reward: -121.97084211102474
Episode 1700 Average Reward: -119.83062276859913
Episode 1800 Average Reward: -119.21890848276651
Episode 1900 Average Reward: -121.55874736726733
Episode 2000 Average Reward: -119.973520157679

In [ ]:
plt.plot(100 * (np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward per Episodes')

# Testing

In [13]:
def load_data():
    Q_Table_flat = np.loadtxt("data.csv", delimiter=",")
    size_states = stateSpace(env)
    Q_Table_shape = (size_states[0], size_states[1], env.action_space.n)
    Qpts = Q_Table_flat.reshape(*Q_Table_shape)
    return Qpts

In [14]:
def testing(env, Q_Table):
    if Q_Table == None:
        Q_Table = load_data()
    env.reset()
    state_adj = lower_state(env)
    done = False
    while not done:
        env.render()
        action = np.argmax(Q_Table[state_adj[0], state_adj[1]])
        next_state, reward, done, _ = env.step(action)
        state_adj = (next_state - env.observation_space.low) * np.array([10, 50])
        state_adj = np.round(state_adj, 0).astype(int)
    env.close()

In [19]:
env.reset()
testing(env, Qpts)